In [2]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.neighbors import NearestNeighbors
from sklearn.impute import SimpleImputer
import numpy as np
import re

# 데이터 로드
df = pd.read_csv('../data/Clean_Red_data.csv')

# 필요한 열만 선택
features = ['wine_name', 'body', 'texture', 'sweetness', 'acidity', 'flavor1', 'flavor2', 'flavor3']
df_features = df[features]

# 숫자형과 문자열형 열을 구분합니다.
num_features = ['body', 'texture', 'sweetness', 'acidity']
cat_features = ['flavor1', 'flavor2', 'flavor3']

# 숫자형과 문자열형 열을 위한 변환기 정의
num_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),  # 결측값 처리
    ('scaler', StandardScaler())
])

cat_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),  # 결측값 처리
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))  # 밀집 행렬로 변환
])

# 열을 변환할 수 있는 ColumnTransformer 정의
preprocessor = ColumnTransformer(
    transformers=[
        ('num', num_transformer, num_features),
        ('cat', cat_transformer, cat_features)
    ]
)

# 전체 데이터 전처리
X = preprocessor.fit_transform(df_features.drop('wine_name', axis=1))

# NearestNeighbors 모델 생성 (n_neighbors를 20으로 설정)
nbrs = NearestNeighbors(n_neighbors=20, algorithm='auto').fit(X)

# 특성 가중치 정의 (예: body, texture, sweetness, acidity에 각각 2배의 가중치 부여)
feature_weights = {'body': 0.5, 'texture': 0.5, 'sweetness': 0.5, 'acidity': 0.5}

def apply_weights(X, feature_weights, num_features):
    # 숫자형 특성의 인덱스를 찾습니다.
    num_feature_indices = [num_features.index(f) for f in feature_weights.keys()]
    X_weighted = X.copy()
    
    # 각 특성에 가중치를 적용합니다.
    for idx, feature in zip(num_feature_indices, feature_weights.keys()):
        X_weighted[:, idx] *= feature_weights[feature]
    
    return X_weighted

def find_similar_wine(wine_name):
    # 입력 와인의 특성값을 찾습니다.
    input_wine = df[df['wine_name'] == wine_name]
    
    if input_wine.empty:
        return "해당 와인을 찾을 수 없습니다."
    
    input_wine_features = input_wine[features[1:]]
    
    # 입력 와인의 특성값을 전처리합니다.
    input_wine_features_transformed = preprocessor.transform(input_wine_features)
    input_wine_features_weighted = apply_weights(input_wine_features_transformed, feature_weights, num_features)
    
    # 전체 데이터에서 가중치를 적용합니다.
    X_weighted = apply_weights(X, feature_weights, num_features)
    
    # 가장 유사한 와인을 찾습니다.
    nbrs_weighted = NearestNeighbors(n_neighbors=20, algorithm='auto').fit(X_weighted)
    distances, indices = nbrs_weighted.kneighbors(input_wine_features_weighted)
    
    # 유사한 와인 목록을 준비합니다.
    similar_wines = []
    for i in range(len(indices[0])):
        wine_index = indices[0][i]
        if wine_index == df.index[df['wine_name'] == wine_name][0]:
            continue  # 자기 자신을 제외
        # 유사도 점수를 거리의 역수로 계산합니다.
        similarity_score = 1 / (distances[0][i] + 1e-5)  # 1e-5는 0으로 나누는 것을 방지합니다.
        # 유사성 점수가 4 이상인 경우 제외
        if similarity_score >= 4:
            continue
        similar_wines.append((df.iloc[wine_index]['wine_name'], similarity_score, distances[0][i]))

    # 유사도 순서대로 정렬
    similar_wines.sort(key=lambda x: x[1], reverse=True)
    
    # 20개 후보 와인 선택
    top_20_wines = similar_wines[:20]

    # 와인 이름에서 연도를 제거하고, 기본 이름을 기준으로 가장 유사한 결과만 남기기
    def remove_year(wine_name):
        return re.sub(r'\b\d{4}\b', '', wine_name).strip()
    
    # 기본 이름을 기준으로 가장 유사한 결과만 남기기
    filtered_wines = {}
    for wine_name, score, distance in top_20_wines:
        base_name = remove_year(wine_name)
        if base_name not in filtered_wines:
            filtered_wines[base_name] = (wine_name, score, distance)
        else:
            # 기존 와인과 비교하여 거리가 더 가까운 경우 업데이트
            if distance < filtered_wines[base_name][2]:
                filtered_wines[base_name] = (wine_name, score, distance)

    # 결과를 리스트로 변환
    filtered_wines_list = sorted(filtered_wines.values(), key=lambda x: x[1], reverse=True)

    # 기본 이름별로 선택된 와인 이름 목록
    selected_wine_names = {wine_name for wine_name, _, _ in filtered_wines_list}

    # 전체 유사도 목록에서 추가 와인 선택
    additional_wines = [ (wine, score, distance) for wine, score, distance in similar_wines if wine not in selected_wine_names]
    
    # 부족한 개수만큼 추가 와인 선택
    if len(filtered_wines_list) < 10:
        additional_wines = additional_wines[:10 - len(filtered_wines_list)]
        final_wines = filtered_wines_list + additional_wines
    else:
        final_wines = filtered_wines_list

    return final_wines[:10]  # 상위 10개 와인 반환

# 사용 예
wine_to_search = 'Pinot Noir 2021'
top_similar_wines = find_similar_wine(wine_to_search)
print(f"가장 유사한 와인들:")
for wine, similarity_score, distance in top_similar_wines:
    print(f"와인: {wine}, 유사성 점수: {similarity_score:.4f}, 거리: {distance:.4f}")


가장 유사한 와인들:
와인: Juliénas 2011, 유사성 점수: 1.0348, 거리: 0.9663
와인: Chianti Castaldo 2003, 유사성 점수: 0.8427, 거리: 1.1867
와인: Mencía 2014, 유사성 점수: 0.8218, 거리: 1.2168
와인: Gamay 2022, 유사성 점수: 0.7378, 거리: 1.3554
와인: Evenstad Reserve Pinot Noir 2006, 유사성 점수: 0.7065, 거리: 1.4154
와인: Pinot Noir 2023, 유사성 점수: 0.7046, 거리: 1.4192
와인: Bella Reserve 2021, 유사성 점수: 0.7016, 거리: 1.4252
와인: Retrospect Pinot Noir 2018, 유사성 점수: 0.7013, 거리: 1.4259
와인: Reserve Pinot Noir 2020, 유사성 점수: 0.7010, 거리: 1.4265
와인: Estate Pinot Noir 2007, 유사성 점수: 0.7010, 거리: 1.4266
